<h1> Mandatory assignment 1 - group 14</h1>

<p>Elisabeth Mathisen S381128</p>
<p>Per-Erik Glærum Grønvik S385710</p>
<p>Umeshgaar Balasubramaniam S385698</p>

Do the following:
1. Pick one use case (defined below).
    We picked the Ruter case.
2. Explore and research which algorithm would work best for this use case (regression or classification)
    To determine the number of passengers on a bus, regression is the best suited algorithm.
3. Document your findings in a markdown cell (3-5 lines) on why you chose this algorithm.
    Classification, gives yes or no answers. This could theoretically be used with some logic, but regression is better suited for this task as it will give the answer directly. The other is however theoretically equally accurate, but practically very inferior.
4. Train the algorithm using Python
    
5. Keep the solution as simple as possible. We are not looking for the best machine-learning algorithm. We are interested in seeing that you know how to work with machine learning.

6. Turn in a **JUPYTER NOTEBOOK** on canvas.

Ruter assignment:

- Predict passenger data for Ruter.

- Use Ruter-data.csv dataset (in data folder). 

- I want you to make a prediction algorithm which predicts the number of passengers on a specific date for a specific bus (pick any one). 

- Input should be date and output will be number of passengers. 

- You should also show the prediction percentage score. 

- Data file: Ruter_data.csv 

The criteria of the task was changed in class:

- Does the data fit into a sigmoid function? Decide if the bus have high or low amount of passengers.

- Test hvor full bussen er med algoritmen: høy / lav.

- Some cost functions give better prediction scores than others. Test many.


In [66]:
# imports

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.api as sm
from datetime import date
from scipy.special import expit
from sklearn.metrics import precision_score
from statsmodels.tools.sm_exceptions import PerfectSeparationWarning
import warnings
warnings.filterwarnings("ignore", category=PerfectSeparationWarning)
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC


Load the data from the file and explore it.

In [67]:
ruter = pd.read_csv('./data/Ruter-data.csv' , sep=';', index_col=None)

## Number of rows and columns in ruter
print("Columns, rows")
print(ruter.shape)
#ruter.info()

Columns, rows
(6000, 17)


In [68]:
ruter.head(10)

,TurId,Dato,Fylke,Område,Kommune,Holdeplass_Fra,Holdeplass_Til,Linjetype,Linjefylke,Linjenavn,Linjeretning,Tidspunkt_Faktisk_Ankomst_Holdeplass_Fra,Tidspunkt_Faktisk_Avgang_Holdeplass_Fra,Tidspunkt_Planlagt_Ankomst_Holdeplass_Fra,Tidspunkt_Planlagt_Avgang_Holdeplass_Fra,Kjøretøy_Kapasitet,Passasjerer_Ombord
0,15006-2020-08-10T10:24:00+02:00,10/08/2020,Viken,Vest,Bærum,Nordliveien,Tjernsmyr,Lokal,Viken,150,0,10:53:53,10:53:59,10:53:00,10:53:00,112,5
1,15002-2020-08-15T12:54:00+02:00,15/08/2020,Viken,Vest,Bærum,Nadderud stadion,Bekkestua bussterminal (Plattform C),Lokal,Viken,150,0,13:12:20,13:12:26,13:12:00,13:12:00,112,5
2,15004-2020-08-03T09:54:00+02:00,03/08/2020,Viken,Vest,Bærum,Ringstabekkveien,Skallum,Lokal,Viken,150,0,10:18:56,10:19:21,10:19:00,10:19:00,112,6
3,15003-2020-07-27T13:00:00+02:00,27/07/2020,Viken,Vest,Bærum,Gruvemyra,Gullhaug,Lokal,Viken,150,1,13:52:04,13:52:26,13:51:00,13:51:00,112,10
4,15002-2020-08-27T07:15:00+02:00,27/08/2020,Viken,Vest,Bærum,Lysaker stasjon (Plattform A),Tjernsmyr,Lokal,Viken,150,1,07:34:13,07:34:53,07:33:00,07:33:00,112,10
5,3110-2020-08-01T16:16:00+02:00,01/08/2020,Oslo,Nordøst,Bjerke,Veitvet (mot Kalbakken),Rødtvet (mot Kalbakken),Lokal,Oslo,31,0,17:22:56,17:23:23,17:16:00,17:16:00,151,8
6,15010-2020-07-28T13:09:00+02:00,28/07/2020,Viken,Vest,Bærum,Nedre Toppenhaug,Øvre Toppenhaug,Lokal,Viken,150,0,13:19:00,13:19:05,13:17:00,13:17:00,112,1
7,15003-2020-07-27T06:18:00+02:00,27/07/2020,Oslo,Indre By,St.Hanshaugen,Hammersborggata (ved Storgata retning vest),St. Olavs plass (mot Frederiks gate),Lokal,Viken,150,1,06:20:24,06:20:29,06:20:00,06:20:00,112,-1
8,21002-2020-08-16T14:33:00+02:00,16/08/2020,Viken,Vest,Bærum,Stein gård,Knabberudveien,Lokal,Viken,150,1,15:10:12,15:10:34,15:09:00,15:09:00,112,2
9,15002-2020-08-13T18:09:00+02:00,13/08/2020,Viken,Vest,Bærum,Bekkestua bussterminal (Plattform C),Stabekk skole,Lokal,Viken,150,0,18:37:41,18:38:19,18:29:00,18:29:00,112,4


In [69]:
#print column names for copy paste purposes
print(ruter.columns.tolist())

['TurId', 'Dato', 'Fylke', 'Område', 'Kommune', 'Holdeplass_Fra', 'Holdeplass_Til', 'Linjetype', 'Linjefylke', 'Linjenavn', 'Linjeretning', 'Tidspunkt_Faktisk_Ankomst_Holdeplass_Fra', 'Tidspunkt_Faktisk_Avgang_Holdeplass_Fra', 'Tidspunkt_Planlagt_Ankomst_Holdeplass_Fra', 'Tidspunkt_Planlagt_Avgang_Holdeplass_Fra', 'Kjøretøy_Kapasitet', 'Passasjerer_Ombord']


In [70]:
# Checking for missing values
print(ruter.isnull().sum())


TurId                                        0
Dato                                         0
Fylke                                        0
Område                                       0
Kommune                                      0
Holdeplass_Fra                               0
Holdeplass_Til                               0
Linjetype                                    0
Linjefylke                                   0
Linjenavn                                    0
Linjeretning                                 0
Tidspunkt_Faktisk_Ankomst_Holdeplass_Fra     0
Tidspunkt_Faktisk_Avgang_Holdeplass_Fra      0
Tidspunkt_Planlagt_Ankomst_Holdeplass_Fra    0
Tidspunkt_Planlagt_Avgang_Holdeplass_Fra     0
Kjøretøy_Kapasitet                           0
Passasjerer_Ombord                           0
dtype: int64


In [71]:
# I turn the passengers who are less than zero to zero, because i assume that the best guess is that this bus is empty. If i wanted to really predict the number i guess i would have to check video of some of these and see if there was a average.
ruter['Passasjerer_Ombord'] = ruter['Passasjerer_Ombord'].apply(lambda x: max(x, 0))
ruter.describe()
ruter.head(10)


,TurId,Dato,Fylke,Område,Kommune,Holdeplass_Fra,Holdeplass_Til,Linjetype,Linjefylke,Linjenavn,Linjeretning,Tidspunkt_Faktisk_Ankomst_Holdeplass_Fra,Tidspunkt_Faktisk_Avgang_Holdeplass_Fra,Tidspunkt_Planlagt_Ankomst_Holdeplass_Fra,Tidspunkt_Planlagt_Avgang_Holdeplass_Fra,Kjøretøy_Kapasitet,Passasjerer_Ombord
0,15006-2020-08-10T10:24:00+02:00,10/08/2020,Viken,Vest,Bærum,Nordliveien,Tjernsmyr,Lokal,Viken,150,0,10:53:53,10:53:59,10:53:00,10:53:00,112,5
1,15002-2020-08-15T12:54:00+02:00,15/08/2020,Viken,Vest,Bærum,Nadderud stadion,Bekkestua bussterminal (Plattform C),Lokal,Viken,150,0,13:12:20,13:12:26,13:12:00,13:12:00,112,5
2,15004-2020-08-03T09:54:00+02:00,03/08/2020,Viken,Vest,Bærum,Ringstabekkveien,Skallum,Lokal,Viken,150,0,10:18:56,10:19:21,10:19:00,10:19:00,112,6
3,15003-2020-07-27T13:00:00+02:00,27/07/2020,Viken,Vest,Bærum,Gruvemyra,Gullhaug,Lokal,Viken,150,1,13:52:04,13:52:26,13:51:00,13:51:00,112,10
4,15002-2020-08-27T07:15:00+02:00,27/08/2020,Viken,Vest,Bærum,Lysaker stasjon (Plattform A),Tjernsmyr,Lokal,Viken,150,1,07:34:13,07:34:53,07:33:00,07:33:00,112,10
5,3110-2020-08-01T16:16:00+02:00,01/08/2020,Oslo,Nordøst,Bjerke,Veitvet (mot Kalbakken),Rødtvet (mot Kalbakken),Lokal,Oslo,31,0,17:22:56,17:23:23,17:16:00,17:16:00,151,8
6,15010-2020-07-28T13:09:00+02:00,28/07/2020,Viken,Vest,Bærum,Nedre Toppenhaug,Øvre Toppenhaug,Lokal,Viken,150,0,13:19:00,13:19:05,13:17:00,13:17:00,112,1
7,15003-2020-07-27T06:18:00+02:00,27/07/2020,Oslo,Indre By,St.Hanshaugen,Hammersborggata (ved Storgata retning vest),St. Olavs plass (mot Frederiks gate),Lokal,Viken,150,1,06:20:24,06:20:29,06:20:00,06:20:00,112,0
8,21002-2020-08-16T14:33:00+02:00,16/08/2020,Viken,Vest,Bærum,Stein gård,Knabberudveien,Lokal,Viken,150,1,15:10:12,15:10:34,15:09:00,15:09:00,112,2
9,15002-2020-08-13T18:09:00+02:00,13/08/2020,Viken,Vest,Bærum,Bekkestua bussterminal (Plattform C),Stabekk skole,Lokal,Viken,150,0,18:37:41,18:38:19,18:29:00,18:29:00,112,4


predict the number of passengers on a specific date, for a specific bus (pick any one). 


Input should be date and output will be number of passengers. You should also show the prediction percentage score.

In [72]:
# just to make sure i have enough data I will start by counting rows for routes Holdeplass_Fra and Holdeplass_Til
# kombinerer fra og til.
ruter['Holdeplass_Fra_Til'] = 'fra_' + ruter['Holdeplass_Fra'] + '_til_' + ruter['Holdeplass_Til']
ruter['Holdeplass_Fra_Til'].value_counts().head(10)


Holdeplass_Fra_Til
fra_Jenseberget_til_Sagdalen                                                  26
fra_Teisenkrysset  (fra Helsfyr)_til_Ulvenkrysset  (fra Helsfyr)              25
fra_Kloppaveien_til_Ahus                                                      21
fra_Vålerenga  (mot Galgeberg)_til_Galgeberg  (fra Etterstad)                 20
fra_Knatten (Solheimsveien)_til_Kloppaveien                                   20
fra_Lillestrøm bussterminal (Plattform 18)_til_Nittedalsgata (mot Kjeller)    19
fra_Karihaugen  (mot Furuset)_til_Folkvangveien  (mot Furuset)                18
fra_Trosterud  (E6 mot Helsfyr)_til_Ulvenkrysset  (mot Helsfyr)               18
fra_Strømsbergveien_til_Stasjonsveien                                         17
fra_Knatten (Solheimsveien)_til_Lørenskog sentrum (plattform 4)               17
Name: count, dtype: int64

In [73]:
# We chose the route fra_Jenseberget_til_Sagdalen and dropped all other rows
ruter = ruter[ruter['Holdeplass_Fra_Til'] == 'fra_Jenseberget_til_Sagdalen']
# Very many variables are not relevant. I will drop them. The Assignment say ass simple ass possible and only date in, number out.
ruter = ruter.drop(columns=['Holdeplass_Fra_Til','TurId', 'Fylke', 'Område', 'Kommune', 'Holdeplass_Fra', 'Holdeplass_Til', 'Linjetype', 'Linjefylke', 'Linjenavn', 'Linjeretning', 'Tidspunkt_Faktisk_Ankomst_Holdeplass_Fra', 'Tidspunkt_Faktisk_Avgang_Holdeplass_Fra', 'Tidspunkt_Planlagt_Ankomst_Holdeplass_Fra', 'Tidspunkt_Planlagt_Avgang_Holdeplass_Fra', 'Kjøretøy_Kapasitet'])
ruter.head(1)


,Dato,Passasjerer_Ombord
156,15/06/2020,6


From assignment:
- Input should be date and output will be number of passengers. 

Interpretation:
- For the model to understand dates it needs to be put in a context. This is a regression problem, and if we split it by weekend it becomes a binary classification problem. So I assume that either the days of week but then it would be a multiclass classification problem. The only way to make it regression is to make a timeseries, as maybe the amount of passengers grows linearly. I find it hard to understand these poorly written assignments. But I guess linearly will capture a trend, but is the data good enough to do that, because weekdays would bring noise. The model should take in a date and put out a date so there is no other interpretations.

In [74]:
# convert date to datetime
ruter['Dato'] = pd.to_datetime(ruter['Dato'], format='%d/%m/%Y')
# first day
first_day = ruter['Dato'].min()
# create new column for day numbers
ruter['day_number'] = (ruter['Dato'] - first_day).dt.days
ruter= ruter.drop(columns=['Dato'])
ruter.head(1)

,Passasjerer_Ombord,day_number
156,6,13


In [75]:
#regression plot X all columns except Passasjerer_Ombord
X = ruter.drop(columns=['Passasjerer_Ombord'])
y = ruter['Passasjerer_Ombord']

model = sm.OLS(y, sm.add_constant(X)).fit()
X = sm.add_constant(X)

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:     Passasjerer_Ombord   R-squared:                       0.009
Model:                            OLS   Adj. R-squared:                 -0.032
Method:                 Least Squares   F-statistic:                    0.2222
Date:               tir., 14 okt 2025   Prob (F-statistic):              0.642
Time:                        21:16:18   Log-Likelihood:                -86.456
No. Observations:                  26   AIC:                             176.9
Df Residuals:                      24   BIC:                             179.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          7.5885      2.412      3.146      0.0

In [76]:
# prediction form date
date = '13/10/2020' #actual date
date_intermediate = pd.to_datetime(date, format='%d/%m/%Y')
day_number_ml = (date_intermediate - first_day).days


passenger = model.predict([1, day_number_ml])
print(f'Predicted number of passengers on {date} is {passenger[0]}')

Predicted number of passengers on 13/10/2020 is 4.253124465938826


- You should also show the prediction percentage score. 

In [77]:
# Adding the prediction percentage score for a linear regression model does not make sense, because linear regression is not a classification model.

From Assignment:

Does the data fit into a sigmoid function? Decide if the bus have high or low amount of passengers.

Test if the the buss is high / low.
    
Some costfunctions give better prediction scores than others. Test many.

In [78]:
def change_sigmoid_treshold(threshold):


    normalized_passengers = (ruter['Passasjerer_Ombord'] - ruter['Passasjerer_Ombord'].mean()) / ruter['Passasjerer_Ombord'].std()
    sigmoid_values = expit(normalized_passengers)


    ruter['high_passengers'] = (sigmoid_values > threshold).astype(int)
    return

change_sigmoid_treshold(0.8)
ruter.head(10)

,Passasjerer_Ombord,day_number,high_passengers
156,6,13,0
320,0,55,0
401,5,21,0
801,16,7,0
850,6,0,0
880,18,58,1
904,16,2,0
1022,13,18,0
1103,3,82,0
1130,5,8,0


In [79]:
#use a classification model not regression, to predict high / low passengers and return precision_score
def classify():
    GLM = sm.GLM(ruter['high_passengers'], X, family=sm.families.Binomial())
    res = GLM.fit()
    ruter['high_passengers_pred'] = (res.predict(X) > 0.5).astype(int)
    return precision_score(ruter["high_passengers"], ruter["high_passengers_pred"], zero_division=0)

print(classify())

0.0


In [80]:
for threshold in np.arange(0.1, 1.0, 0.05):
    change_sigmoid_treshold(threshold)
    print(f"different probability tresholds: {threshold:.2f} {classify():.3f}")


different probability tresholds: 0.10 1.000
different probability tresholds: 0.15 1.000
different probability tresholds: 0.20 1.000
different probability tresholds: 0.25 1.000
different probability tresholds: 0.30 0.727
different probability tresholds: 0.35 0.591
different probability tresholds: 0.40 0.688
different probability tresholds: 0.45 0.636
different probability tresholds: 0.50 0.000
different probability tresholds: 0.55 0.000
different probability tresholds: 0.60 0.000
different probability tresholds: 0.65 0.000
different probability tresholds: 0.70 0.000
different probability tresholds: 0.75 0.000
different probability tresholds: 0.80 0.000
different probability tresholds: 0.85 0.000
different probability tresholds: 0.90 0.000
different probability tresholds: 0.95 0.000


<h3>Test many cost functions (different algorithms) and compare prediction scores</h3>

In [ ]:


# Use optimal threshold from previous testing
change_sigmoid_treshold(0.3)

y_true = ruter['high_passengers'].values
X_clean = X.iloc[:, 1:]  # Remove intercept for sklearn

print("Testing many cost functions (algorithms):\n")

# 1. GLM families
families = [
    ('GLM Binomial (logit)', sm.families.Binomial()),
    ('GLM Binomial (probit)', sm.families.Binomial(link=sm.families.links.Probit())),
    ('GLM Poisson', sm.families.Poisson()),
    ('GLM Gaussian (OLS)', sm.families.Gaussian()),
]

for name, family in families:

    glm = sm.GLM(ruter['high_passengers'], X, family=family)
    res = glm.fit()
    y_pred = (res.predict(X) > 0.5).astype(int)
    prec = precision_score(y_true, y_pred, zero_division=0)
    print(f"{name:35s} Precision: {prec:.3f}")
 

# 2. Logistic Regression (different solvers = different optimization)
print(f"{'Logistic Regression (lbfgs)':35s} Precision: {precision_score(y_true, LogisticRegression(solver='lbfgs', max_iter=1000).fit(X_clean, y_true).predict(X_clean), zero_division=0):.3f}")
print(f"{'Logistic Regression (liblinear)':35s} Precision: {precision_score(y_true, LogisticRegression(solver='liblinear').fit(X_clean, y_true).predict(X_clean), zero_division=0):.3f}")

# 3. SVM (different kernels)
print(f"{'SVM (linear)':35s} Precision: {precision_score(y_true, SVC(kernel='linear').fit(X_clean, y_true).predict(X_clean), zero_division=0):.3f} ")
print(f"{'SVM (rbf)':35s} Precision: {precision_score(y_true, SVC(kernel='rbf').fit(X_clean, y_true).predict(X_clean), zero_division=0):.3f}")


Testing many cost functions (algorithms):

GLM Binomial (logit)                Precision: 0.727
GLM Binomial (probit)               Precision: 0.727
GLM Poisson                         Precision: 0.708
GLM Gaussian (OLS)                  Precision: 0.708
Logistic Regression (lbfgs)         Precision: 0.727
Logistic Regression (liblinear)     Precision: 0.731
SVM (linear)                        Precision: 0.731 
SVM (rbf)                           Precision: 0.857


GLM Binomial (logit)                Precision: 0.727
GLM Binomial (probit)               Precision: 0.727
GLM Poisson                         Precision: 0.708
GLM Gaussian (OLS)                  Precision: 0.708
Logistic Regression (lbfgs)         Precision: 0.727
Logistic Regression (liblinear)     Precision: 0.731
SVM (linear)                        Precision: 0.731 
SVM (rbf)                           Precision: 0.857


Among all tested models, the SVM with an RBF kernel achieved the highest precision (0.857), indicating it best distinguished high-passenger cases from low-passenger ones. 

The logistic and linear models (GLM logit, probit, and linear SVM) produced nearly identical precision scores around 0.73, which is expected since they assume a linear decision boundary. 

The Poisson and Gaussian GLMs performed slightly worse (≈0.71), reflecting that their distributional assumptions do not fit a binary outcome well. 

The RBF kernel’s nonlinearity allows it to model more complex relationships, explaining its superior precision.